In [1]:
import pandas as pd
import geopandas as gpd
# from tabula import read_pdf

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# df_informacoes = read_pdf('data/censo_2010/BASE DE INFORMAåES POR SETOR CENSITµRIO Censo 2010 - Universo.pdf', stream=True, pages=46, lattice=True)

In [3]:
# df_informacoes[0]

In [4]:
## Malha de setores do BR
# https://www.ibge.gov.br/geociencias/downloads-geociencias.html?caminho=organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/2021/Malha_de_setores_(shp)_Brasil

In [5]:
## Download das tabelas e documentacao
# https://www.ibge.gov.br/estatisticas/sociais/populacao/9662-censo-demografico-2010.html?edicao=10410

# Selecao das variáveis

## Habitação einfraestrutura

* Porcentagem de domicílios do tipo cômodo
* Porcentagem de domicílios sem coleta de lixo
* Porcentagem de domicílios sem ligação à rede de abastecimento de água
* Porcentagem de domicílios sem banheiros ou sanitários
* Porcentagem de domicílios sem ligação à rede de esgoto ou fossa séptica
* Porcentagem de domicílios – outra forma de posse da moradia
* Número de banheiros por habitante

## Renda e escolaridade do responsável pelo domicílio

* Porcentagem de responsáveis por domicílio não alfabetizados
* Porcentagem de responsáveis por domicílio com menos de 30 anos não alfabetizados
* Porcentagem de responsáveis por domicílio com renda de até 3 salários mínimos
* Porcentagem de responsáveis por domicílio com menos de 8 anos de estudo
* Anos médios de estudo do responsável pelo domicílio
* Renda média do responsável pelo domicílio

## Aspectos demográficos

* Número de domicílios particulares permanentes no setor censitário
* Número de domicílios improvisados no setor censitário
* Número de pessoas residentes no setor censitário
* Porcentagem de responsáveis por domicílios com menos de 30 anos
* Número médio de pessoas por domicílio



In [6]:
gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2021.zip')

In [7]:
gdf_br.shape

(452248, 13)

In [8]:
gdf_br.NM_SIT.value_counts()

Área Urbana de Alta Densidade de Edificações     306095
Área Rural (exclusive aglomerados)                83569
Área Urbana de Baixa Densidade de Edificações     31869
Povoado                                           16832
Núcleo Urbano                                      7682
Lugarejo                                           5010
Massa de Água                                       894
Núcleo Rural                                        295
Name: NM_SIT, dtype: int64

In [9]:
gdf_br.columns

Index(['CD_SETOR', 'CD_SIT', 'NM_SIT', 'CD_UF', 'NM_UF', 'SIGLA_UF', 'CD_MUN',
       'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST', 'geometry'],
      dtype='object')

In [10]:
UFS = [
    'AC',
    'AL',
    'AM',
    'AP',
    'BA',
    'CE',
    'DF',
    'ES',
    'GO',
    'MA',
    'MG',
    'MS',
    'MT',
    'PA',
    'PB',
    'PE',
    'PI',
    'PR',
    'RJ',
    'RN',
    'RO',
    'RR',
    'RS',
    'SC',
    'SE',
    'SP1',
    'SP2',
    'TO'
]

In [11]:
## Hash para montar a base de dados
# {'file1': ['var1', 'var2', ..., 'var n'], ...,'file n': ['var1', 'var2', ..., 'var n']}

model_vars = {
    'Basico':{
        # 'Cod_Grandes Regiões': 'Código das GRandes REgiões',
        # 'Nome_Grande_Regiao': 'Nome GRande Região',
        'Cod_UF': 'uf',
        'Cod_RM': 'rm',
        'Nome_da_RM': 'nome_rm',
        'Cod_municipio': 'cod_municipio',
        'Nome_do_municipio': 'nome_municipio',
        'Situacao_setor': 'situacao_setor',
        'Tipo_setor': 'tipo_do_setor',
        'V001': 'numero_de_domicilios',
        'V002': 'numero_moradores'
    },
    'Domicilio01': {
        'V003': 'domicilios_tipo_casa',
        'V002': 'domicilios_particulares_permanentes',
        'V035': 'domicilios_com_lixo_coletado',
        'V106': 'domicilios_tipo_casa_com_abastecimento_de_agua',
        'V023': 'domicilios_particulares_permanentes_sem_banheiro_ou_sanitario',
        'V017': 'domicilios_particulares_permanentes_com_banheiro_ligado_a_esgoto',
        'V018': 'domicilios_particulares_permanentes_com_banheiro_ligado_a_fossa_septica',
        'V025': 'domicilios_particulares_permanentes_com_1_banheiro',
        'V026': 'domicilios_particulares_permanentes_com_2_banheiros',
        'V027': 'domicilios_particulares_permanentes_com_3_banheiros',
        'V028': 'domicilios_particulares_permanentes_com_4_banheiros',
        'V029': 'domicilios_particulares_permanentes_com_5_banheiros',
        'V030': 'domicilios_particulares_permanentes_com_6_banheiros',
        'V031': 'domicilios_particulares_permanentes_com_7_banheiros',
        'V032': 'domicilios_particulares_permanentes_com_8_banheiros',
        'V033': 'domicilios_particulares_permanentes_com_mais_de_9_banheiros',
        'V010': 'domicilios_particulares_permanentes_cedidos_de_outra_forma',
        'V011': 'domicilios_particulares_permanentes_em_outra_condicao_de_ocupacao',
    },
    'Responsavel02': {
        'V001': 'pessoas_responsaveis',
        'V002': 'pessoas_responsaveis_com_10_anos',
        'V003': 'pessoas_responsaveis_com_11_anos',
        'V004': 'pessoas_responsaveis_com_12_anos',
        'V005': 'pessoas_responsaveis_com_13_anos',
        'V006': 'pessoas_responsaveis_com_14_anos',
        'V007': 'pessoas_responsaveis_com_15_anos',
        'V008': 'pessoas_responsaveis_com_16_anos',
        'V009': 'pessoas_responsaveis_com_17_anos',
        'V010': 'pessoas_responsaveis_com_18_anos',
        'V011': 'pessoas_responsaveis_com_19_anos',
        'V012': 'pessoas_responsaveis_com_20_anos',
        'V013': 'pessoas_responsaveis_com_21_anos',
        'V014': 'pessoas_responsaveis_com_22_anos',
        'V015': 'pessoas_responsaveis_com_23_anos',
        'V016': 'pessoas_responsaveis_com_24_anos',
        'V017': 'pessoas_responsaveis_com_25_anos',
        'V018': 'pessoas_responsaveis_com_26_anos',
        'V019': 'pessoas_responsaveis_com_27_anos',
        'V020': 'pessoas_responsaveis_com_28_anos',
        'V021': 'pessoas_responsaveis_com_29_anos',
        'V093': 'pessoas_responsaveis_alfabetizadas',
        'V094': 'pessoas_responsaveis_alfabetizadas_de_10_a_14_anos',
        'V095': 'pessoas_responsaveis_alfabetizadas_de_15_a_19_anos',
        'V096': 'pessoas_responsaveis_alfabetizadas_de_20_a_24_anos',
        'V097': 'pessoas_responsaveis_alfabetizadas_de_25_a_29_anos'
    },
    'DomicilioRenda':{
        'V001': 'total_domicilios_improvisados',
        'V002': 'total_rendimento_dos_domicilios',
    },
    'ResponsavelRenda':{
        'V001': 'responsaveis_com_renda_ate_meio_salario_minimo',
        'V002': 'responsaveis_com_renda_de_meio_a_1_salario_minimo',
        'V003': 'responsaveis_com_renda_de_1_a_2_salarios_minimos',
        'V004': 'responsaveis_com_renda_de_2_a_3_salarios_minimos'
    }
}


In [23]:
dfs = []
for uf in UFS:
    print(f'Processando {uf}')
    dfs_uf = []
    for file in model_vars:
        file_path = f'data/censo_2010/csvs/{file}_{uf}.csv'
        fields = list(model_vars[file].keys())
        fields.append('Cod_setor')
        try:
            df = pd.read_csv(file_path, delimiter=';', usecols=fields, dtype="string", encoding='UTF-8').set_index('Cod_setor')
        except:
            try:
                df = pd.read_csv(file_path, delimiter=';', usecols=fields, dtype="string", encoding='ISO-8859-1').set_index('Cod_setor')
            except:
                df = pd.read_csv(file_path, delimiter=',', usecols=fields, dtype="string", encoding='UTF-8').set_index('Cod_setor')
        df.rename(columns=model_vars[file], inplace=True)
        dfs_uf.append(df)
        # break
    df = pd.concat(dfs_uf, axis=1)
    dfs.append(df)

Processando AC
Processando AL
Processando AM
Processando AP
Processando BA
Processando CE
Processando DF
Processando ES
Processando GO
Processando MA
Processando MG
Processando MS
Processando MT
Processando PA
Processando PB
Processando PE
Processando PI
Processando PR
Processando RJ
Processando RN
Processando RO
Processando RR
Processando RS
Processando SC
Processando SE
Processando SP1
Processando SP2
Processando TO


In [24]:
df = pd.concat(dfs)

In [40]:
df.nome_rm.value_counts()

Municípios não pertencentes a estrutura de RM        173563
RM São Paulo                                          29943
RM Rio de Janeiro                                     19507
RM Belo Horizonte                                      8523
RM Porto Alegre                                        6971
RIDE - Região Integrada de Desenvolvimento do ...      6059
RM Salvador                                            5023
RM Recife                                              4535
RM Fortaleza                                           4480
RM Curitiba                                            4348
RM Campinas                                            4113
RM Baixada Santista                                    2968
RM Manaus                                              2899
RM Goiânia                                             2882
RM Grande Vitória                                      2838
Munic¡pios não pertencentes a estrutura de RM          2089
RM Belém                                

In [27]:
len(df.rm.value_counts().to_list())

41

In [42]:
CAMPOS_MODELO = [
    'uf',
    'rm',
    'nome_rm',
    'cod_municipio',
    'nome_municipio',
    'situacao_setor',
    'tipo_do_setor',
    'porcentagem_domicilios_sem_coleta_de_lixo',
    'porcentagem_domicilios_sem_ligacao_rede_de_agua',
    'porcentagem_domicilios_sem_banheiros',
    'porcentagem_domicilios_sem_esgoto_ou_fossa',
    # 'porcentagem_domicilios_tipo_comodo',
    'porcentagem_domicilios_cedidos_outra_forma',
    'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante',
    'porcentagem_responsaveis_nao_alfabetizado',
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
    'porcentagem_responsaveis_renda_ate_3_salarios',
    # 'porcentagem_responsaveis_menos_8_anos_de_estudo',
    # 'anos_medios_estudo_responsavel',
    'renda_media_responsavel',
    'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados',
    'numero_de_pessoas_residentes',
    'porcentagem_de_responsaveis_com_menos_30_anos',
    'numero_medio_pessoas'
]